# Importing and Transforming raw DATA

In [2]:
import numpy as np
import pandas as pd
import graphlab as gl
import os
from nltk.tokenize import RegexpTokenizer
from nltk.stem import *

folder = os.getcwd() ; print folder

/home/arda/Documents/youtube


In [3]:
import graphlab as gl

In [ ]:
gl.SFrame

In [5]:
#Importing raw data
pd_train = pd.DataFrame.from_csv(folder + '/data/train_sample.csv',sep=',',header=0,index_col=None, encoding='utf-8')
#pd_test = pd.DataFrame.from_csv(folder + '/data/test_sample.csv',sep=';',header=0,index_col=None)

CParserError: Error tokenizing data. C error: Expected 16 fields in line 23582, saw 17


##### Adding some new features 

In [7]:
def RemovePonctuationAndStemming(r):
    new_r = tokenizer.tokenize(r)
    
    stemmer = PorterStemmer()
    
    out = [stemmer.stem(word) for word in new_r]    
    return out


def addStuff(pd):
    pd['lenght'] = pd['Sentence'].apply(lambda r: len(r)).astype(float)
    
    #number of CAPS character
    pd['nb_caps_char'] = pd['Sentence'].apply(lambda r: np.sum([1 for letter in r if letter.isupper()])).astype(float)
    
    #number of CAPS word
    pd['nb_caps_word'] = pd['Sentence'].apply(lambda r: np.sum([1 for word in r.split(" ") if len(word) > 2 if word.isupper()])).astype(float)
    
    #one column for each punctuation token
    pd['nb_?'] = pd['Sentence'].apply(lambda r: np.sum([1 for letter in r if letter=='?'])  )
    
    pd['nb_,'] = pd['Sentence'].apply(lambda r: np.sum([1 for letter in r if letter==','])  )
    
    pd['nb_!'] = pd['Sentence'].apply(lambda r: np.sum([1 for letter in r if letter=='!'])  )
    
    pd['nb_.'] = pd['Sentence'].apply(lambda r: np.sum([1 for letter in r if letter=='.'])  )
    
    pd['nb_;'] = pd['Sentence'].apply(lambda r: np.sum([1 for letter in r if letter==';'])  )
    
    pd["nb_'"] = pd['Sentence'].apply(lambda r: np.sum([1 for letter in r if letter=="'"])  )
    
    pd['nb_all'] = pd['nb_?']+pd['nb_,']+pd['nb_!']+pd['nb_.']+pd['nb_;']+pd["nb_'"]
    
    pd['nb_,;'] = pd['nb_,'] * pd['nb_;']
    
    #diversiy
    pd['diversity'] = pd['Sentence'].apply(lambda r: float(len(np.unique(r.split(" ")))) / float(len(r)))    
    
    return pd

In [8]:
pd_train = addStuff(pd_train)
pd_test = addStuff(pd_test)

In [9]:
tokenizer = RegexpTokenizer(r'\w+')

pd_train['Sentence_stem_punct'] = pd_train['Sentence'].apply(lambda r: ''.join([char for char in r if ord(char)<128]))
pd_test['Sentence_stem_punct'] = pd_test['Sentence'].apply(lambda r: ''.join([char for char in r if ord(char)<128]))

pd_train['Sentence_stem_punct'] = pd_train['Sentence_stem_punct'].apply(lambda r: ' '.join(RemovePonctuationAndStemming(r)) )
pd_test['Sentence_stem_punct'] = pd_test['Sentence_stem_punct'].apply(lambda r: ' '.join(RemovePonctuationAndStemming(r)) )

In [12]:
pd_train['Sentence'].values

array(["What's onkores, Bilgewater?",
       "He wouldn't ever dared to talk such talk in his life before.",
       "Then she got to talking about her husband, and about her relations up the river, and her relations down the river, and about how much better off they used to was, and how they didn't know but they'd made a mistake coming to our town, instead of letting well aloneand so on and so on, till I was afeard I had made a mistake coming to her to find out what was going on in the town; but by and by she dropped on to pap and the murder, and then I was pretty willing to let her clatter right along.",
       ..., 'HAMLET Marry, this is miching mallecho; it means mischief.',
       "LORD POLONIUS At such a time I'll loose my daughter to him: Be you and I behind an arras then; Mark the encounter: if he love her not And be not from his reason fall'n thereon, Let me be no assistant for a state, But keep a farm and carters.",
       "MARCELLUS Let's do't, I pray; and I this morning know

In [30]:
from nltk import word_tokenize, pos_tag
from nltk.tokenize import RegexpTokenizer
import re
import nltk

In [77]:
pd_train['Sentence_tag'] = pd_train['Sentence'].apply(lambda r: ''.join([char for char in r if ord(char)<128]))


In [80]:
pd_test['Sentence_tag'] = pd_test['Sentence'].apply(lambda r: ''.join([char for char in r if ord(char)<128]))

In [74]:
#tokenizer = RegexpTokenizer('\w+|\$[\d\.]+|\S+')
tokenizer = RegexpTokenizer("\w+|[-,.?!;]")
def TagStringwithPunct(r):
    
    tokenized = tokenizer.tokenize(r)
    tagged_list = pos_tag(tokenized)
    
    final_str = []
    for word,tag in tagged_list:
        final_str.append(tag+"_"+word)
    
    
    return ' '.join(final_str)

In [78]:
pd_train['Sentence_tag'] = pd_train['Sentence_tag'].apply(lambda r: TagStringwithPunct(r) )

In [81]:
pd_test['Sentence_tag'] = pd_test['Sentence_tag'].apply(lambda r: TagStringwithPunct(r) )

# Saving munged DATA into a folder

In [89]:
pd_train.to_pickle(folder +'/data_munged/pd_train_munged')
pd_test.to_pickle(folder +'/data_munged/pd_test_munged')